# Dataroma Aggregator

Load Modules

In [56]:
import pandas as pd
import numpy as np
import re
from collections import Counter

Load Investor List

In [57]:
def investorList():
    dfs = pd.read_html('https://www.dataroma.com/m/managers.php',header=0)
    investordf = pd.DataFrame()
    count = 0
    for i in dfs[0]['Portfolio Manager - Firm']:
        count = count + 1
        if ('-' in i) == True:
            tempin = pd.DataFrame([[i.split('-',1)[1].lstrip(),i.split('-',1)[0].rstrip()]],index=[count],columns=['Firm','Investor'])
        else:
            tempin = pd.DataFrame([[i,'None']],index=[count],columns=['Firm','Investor'])
        investordf = pd.concat([investordf,tempin])

    investorlst = ['AIM','AV','GFT','psc','LMM','HRSVX','oaklx','fairx','ic','ARFFX','DJCO','tci','SA','cfimx','AC','tp','abc','GLRE','MAVFX',
                   'WP','AM','DODGX','YAFFX','EL','FPPTX','ca','CCM','aq','SSHFX','oc','FEVAX','LUK','VA','CAAPX','KB','mc','oa','MPGFX',
                   'hcmax','LLPFX','MVALX','SAM','MFP','PI','TF','DA','FFH','pzfvx','OFALX','TORYX','jensx','SEQUX','lmvtx','BAUPOST',
                   'LPC','FPACX','FS','TA','MKL','GR','TVAFX','TWEBX','WVALX','BRK','cc']
    investordf['ext'] = investorlst
    return investordf

In [58]:
lst = investorList()

In [59]:
print(lst.head())

                                    Firm      Investor  ext
1         Atlantic Investment Management  Alex Roepers  AIM
2                Arlington Value Capital  Allan Mecham   AV
3  Bill & Melinda Gates Foundation Trust          None  GFT
4     Pershing Square Capital Management   Bill Ackman  psc
5                  Miller Value Partners   Bill Miller  LMM


Get Investor Holdings

In [19]:
def investorHoldings(ext,investordf):
    ext = 'AIM'
    Wholedf = pd.DataFrame()
    dfs = pd.read_html('https://www.dataroma.com/m/hist/p_hist.php?f='+str(ext),header=0)
    QuarterHoldings = []
    for i in range(len(dfs[0])):
        holdings = []
        for j in list(range(1,20)):
            if ((type(dfs[0]['Top 20 holdings history (left to right).'+str(j)].iloc[i]) != float) and (type(dfs[0]['Top 20 holdings history (left to right).'+str(j)].iloc[i]) != np.float64)):
                if dfs[0]['Portfolio Value'].iloc[i].split(' ',1)[1] == 'B':
                    portvalue = float(dfs[0]['Portfolio Value'].iloc[i].split(' ',1)[0].lstrip('$'))*1000000000
                else:
                    portvalue = float(dfs[0]['Portfolio Value'].iloc[i].split(' ',1)[0].lstrip('$'))*1000000
                if len(dfs[0]['Top 20 holdings history (left to right).'+str(j)].iloc[i].split('%',1)[0].split('.',1)[1]) > 4:
                    q = ''.join(c for c in dfs[0]['Top 20 holdings history (left to right).'+str(j)].iloc[i].split('%',1)[0].split('.',1)[1] if c.isdigit())
                    if len(q) == 3:
                        qq = float(q[0]+'.'+q[1:])
                    else:
                        qq = float(q[:1]+'.'+q[2:])
                else:
                    if len(dfs[0]['Top 20 holdings history (left to right).'+str(j)].iloc[i].split('%',1)[0].split('.',1)[1]) == 2:
                        qq = float('0.'+dfs[0]['Top 20 holdings history (left to right).'+str(j)].iloc[i].split('%',1)[0].split('.',1)[1])
                    else:
                        qq = float(dfs[0]['Top 20 holdings history (left to right).'+str(j)].iloc[i].split('%',1)[0].split('.',1)[1])
                holdings.append([dfs[0]['Top 20 holdings history (left to right).'+str(j)].iloc[i].split(' ',1)[0],int(portvalue * (qq/100))])
            else:
                continue
        QuarterHoldings.append([[dfs[0]['Period'].iloc[i].split(' ', 2)[0]+' '+dfs[0]['Period'].iloc[i].split(' ',2)[2],int(portvalue)],holdings])
    QHdf = pd.DataFrame()
    count = len(QuarterHoldings)
    for i in QuarterHoldings:
        count = count - 1
        ticker = []
        amount = []
        for j in i[1]:
            ticker.append(j[0])
            amount.append(j[1])
 
        tempQHdf = pd.DataFrame(ticker,columns=['TICKER'])
        tempQHdf['AMOUNT'] = amount    
        tempQHdf['DATE'] = [i[0][0]]*len(tempQHdf)
        tempQHdf['PortValue'] = [i[0][1]]*len(tempQHdf)
        tempQHdf['count'] = count
        tempQHdf = tempQHdf.set_index('count',drop=True)
        tempQHdf['%'] = tempQHdf['AMOUNT']/tempQHdf['PortValue']
        QHdf = pd.concat([QHdf,tempQHdf])

    QHdf = QHdf.sort_index()
    QHdf = QHdf.reset_index()
    QHdf['Firm'] = list(investordf[investordf['ext'] == ext]['Firm'])*len(QHdf)
    QHdf['Investor'] = list(investordf[investordf['ext'] == ext]['Investor'])*len(QHdf)
    QHdf['ext'] = list(investordf[investordf['ext'] == ext]['ext'])*len(QHdf)
    Wholedf = pd.concat([Wholedf,QHdf])
    return Wholedf

For this example we will pull Alex Roepers holdings using ext: AIM

In [20]:
holdings = investorHoldings('AIM',lst)

In [22]:
print(holdings.head())

   count TICKER    AMOUNT     DATE   PortValue       %  \
0      0    GEO  13310000  2012 Q1  1210000000  0.0110   
1      0    JOY  17061000  2012 Q1  1210000000  0.0141   
2      0    ENR  12463000  2012 Q1  1210000000  0.0103   
3      0    ASH  22747999  2012 Q1  1210000000  0.0188   
4      0    KMT  18392000  2012 Q1  1210000000  0.0152   

                             Firm      Investor  ext  
0  Atlantic Investment Management  Alex Roepers  AIM  
1  Atlantic Investment Management  Alex Roepers  AIM  
2  Atlantic Investment Management  Alex Roepers  AIM  
3  Atlantic Investment Management  Alex Roepers  AIM  
4  Atlantic Investment Management  Alex Roepers  AIM  


Get the S&P 500 Grid

In [25]:
#S&P 500 Grid
#gridType = ['Ownership','% of total portfolio','Last qtr buys','Last 6 months buys','Last qtr sells','Last 6 months sells']
#returnType = ['List', 'DataFrame']
def sp500Grid(gridType, returnType):
    if gridType == 'Ownership':
        df = pd.read_html('https://www.dataroma.com/m/grid.php')
    elif gridType == '% of total portfolio':
        df = pd.read_html('https://www.dataroma.com/m/grid.php?s=p')
    elif gridType == 'Last qtr buys':    
        df = pd.read_html('https://www.dataroma.com/m/grid.php?s=q')
    elif gridType == 'Last 6 months buys':
        df = pd.read_html('https://www.dataroma.com/m/grid.php?s=h')
    elif gridType == 'Last qtr sells':
        df = pd.read_html('https://www.dataroma.com/m/grid.php?s=sq')
    elif gridType == 'Last 6 months sells':
        df = pd.read_html('https://www.dataroma.com/m/grid.php?s=sh')
    dfn = df[0].stack().reset_index(drop=True)
    dflst = []
    for i in range(len(dfn)):
        temp = dfn.iloc[i]
        ticker = temp.split(' ')[0] #ticker
        try:
            holdprice = re.findall(r'\d+', temp.split(':')[2]) #Hold Price
            holdprice = holdprice[0]+'.'+holdprice[1]
        except:
            holdprice = 0
        dfsn = temp.split("(")[1]
        industry = dfsn.split(")")[0] #Industry
        name = temp.split("(")[0].split(' ')[1:] #Company Name
        namen = ''
        for i in range(len(name)):
            namen = namen +' '+name[i]
        dfsnn = temp.split(':')[1].split(' ')[1]
        ownership = re.findall(r'\d+', dfsnn) #Ownership
        dflst.append({'Ticker': ticker, 'Company': namen, 'Industry': industry, 'Ownership': ownership[0], 'Hold Price': holdprice})
    if returnType == 'List':
        result = dflst
    elif returnType == 'DataFrame':
        result = pd.DataFrame()
        for i in range(len(dflst)):
            finaln = pd.DataFrame(columns=[i],index=['Ticker','Company','Industry','Ownership','Hold Price']).from_dict(dflst[i],orient='index')
            finaln.columns = [i]
            result[[i]] = finaln
    return result 

In [26]:
grid = sp500Grid('Ownership', 'DataFrame')

In [28]:
print(grid)

                       0                       1             2    \
Ticker                  FB                   GOOGL           WFC   
Company      Facebook Inc.           Alphabet Inc.   Wells Fargo   
Industry        Technology  Information Technology    Financials   
Ownership               27                      23            22   
Hold Price          175.35                 1221.35         26.70   

                               3                         4    \
Ticker                        MSFT                     BRK.B   
Company            Microsoft Corp.   Berkshire Hathaway CL B   
Industry    Information Technology                Financials   
Ownership                       21                        20   
Hold Price                  175.79                    182.59   

                               5                      6    \
Ticker                         BAC                    JPM   
Company      Bank of America Corp.   JPMorgan Chase & Co.   
Industry               

Get Grand Portfolio

If pulling 'Sector'. Then you should specify a 'sectorType'.

If you are looking for holdings, set 'sector' and 'sectorType' to 0.

In [45]:
#sector = [0,1] Sector allows you to focus on industry type.
#sectorType = ['Ownership','Qtr buys','Qtr sells','Qtr net','6 month buys','6 month sells','6 month net'] Filter by.
#holdings = ['Holdings','Qtr buys','Qtr sells','6 month buys','6 month sells'] This allows you to choose a filter.
#sort = ['Percent','buys/sells','maxBet','Ownership'] This allows you to sort the results by these variables.
def grandPortfolio(sector, sectorType, holdings, sort):
    if sector == 1:
        if sectorType == 'Ownership':
            df = pd.read_html('https://www.dataroma.com/m/stats/stats.php')
        elif sectorType == 'Qtr buys':
            df = pd.read_html('https://www.dataroma.com/m/stats/stats.php?o=b')
        elif sectorType == 'Qtr sells':
            df = pd.read_html('https://www.dataroma.com/m/stats/stats.php?o=s')            
        elif sectorType == 'Qtr net':
            df = pd.read_html('https://www.dataroma.com/m/stats/stats.php?o=n')               
        elif sectorType == '6 month buys':
            df = pd.read_html('https://www.dataroma.com/m/stats/stats.php?o=bh')            
        elif sectorType == '6 month sells':
            df = pd.read_html('https://www.dataroma.com/m/stats/stats.php?o=sh')             
        elif sectorType == '6 month net':
            df = pd.read_html('https://www.dataroma.com/m/stats/stats.php?o=nh')   
        else:
            df = None
        return df
            
    elif sector == 0:
        if holdings == 'Holdings':
            dfn = pd.DataFrame()
            for i in range(15):    
                try:
                    temp = pd.read_html('https://www.dataroma.com/m/g/portfolio.php?L='+str(i)) #1-15
                    dfn = pd.concat([dfn,temp[0]])
                except:
                    temp = 0
            dfn = dfn.reset_index(drop=True).dropna() 
        elif holdings == 'Qtr buys':
            dfn = pd.DataFrame()
            for i in range(15):    
                try:
                    temp = pd.read_html('https://www.dataroma.com/m/g/portfolio_b.php?q=q&L='+str(i)) #1-15
                    dfn = pd.concat([dfn,temp[0]])
                except:
                    temp = 0
            dfn = dfn.reset_index(drop=True).dropna()             
        elif holdings == 'Qtr sells':
            dfn = pd.DataFrame()
            for i in range(15):    
                try:
                    temp = pd.read_html('https://www.dataroma.com/m/g/portfolio_s.php?q=q&L='+str(i)) #1-15
                    dfn = pd.concat([dfn,temp[0]])
                except:
                    temp = 0
            dfn = dfn.reset_index(drop=True).dropna()               
        elif holdings == '6 month buys':
            dfn = pd.DataFrame()
            for i in range(15):    
                try:
                    temp = pd.read_html('https://www.dataroma.com/m/g/portfolio_b.php?q=h&L='+str(i)) #1-15
                    dfn = pd.concat([dfn,temp[0]])
                except:
                    temp = 0
            dfn = dfn.reset_index(drop=True).dropna()                
        elif holdings == '6 month sells':
            dfn = pd.DataFrame()
            for i in range(15):    
                try:
                    temp = pd.read_html('https://www.dataroma.com/m/g/portfolio_b.php?q=h&L='+str(i)) #1-15
                    dfn = pd.concat([dfn,temp[0]])
                except:
                    temp = 0
            dfn = dfn.reset_index(drop=True).dropna() 
        else:
            dfn = None
              
        dfn = dfn.reset_index(drop=True)
        
        try:
            dfn = dfn[~dfn.Symbol.str.contains("Next")]  
        except:
            pass
        
        #Reset Column Names
        if len(dfn.columns) == 5:
            dfn.columns = ['Symbols','Company','%','buys/sells','Hold Price']
        elif len(dfn.columns) == 6:    
            dfn.columns = ['Symbol','Company','%','Ownership','Hold Price','Max Bet']
        #Sort Values
        if sort == 'Percent':            
            dfn = dfn.sort_values('%',ascending=False)
        elif sort == 'buy/sells':
            dfn = dfn.sort_values('buys/sells',ascending=False)
        elif sort == 'Ownership':
            dfn = dfn.sort_values('Ownership',ascending=False)            
        elif sort == 'Max Bet':
            dfn = dfn.sort_values('Max Bet',ascending=False)              

        return dfn

In [46]:
gp = grandPortfolio(0, 0, 'Qtr buys', 'Percent')

In [48]:
print(gp.head())

   Symbols                           Company      % buys/sells Hold Price
0      RTX       Raytheon Technologies Corp.  0.070          3     $86.84
32     RTX       Raytheon Technologies Corp.  0.070          3     $86.84
33    MSGE  Madison Square Garden Ent. Corp.  0.067          2     $75.00
1     MSGE  Madison Square Garden Ent. Corp.  0.067          2     $75.00
3      JPM              JPMorgan Chase & Co.  0.047          1     $90.84


Get Realtime data

This the most recent transactions from the super investors.

In [49]:
def realTime():
    return pd.read_html('https://www.dataroma.com/m/rt.php')[0]

In [50]:
rt = realTime()

In [52]:
print(rt.head())

  Transaction Date        Filing                   Reporting Name Activity  \
0      07 Aug 2020  07 Aug 20:32                     ICAHN CARL C      Buy   
1      06 Aug 2020  07 Aug 20:32                     ICAHN CARL C      Buy   
2      05 Aug 2020  07 Aug 20:32                     ICAHN CARL C      Buy   
3      03 Aug 2020  05 Aug 16:00  ABRAMS CAPITAL MANAGEMENT, L.P.     Sell   
4      04 Aug 2020  04 Aug 20:59           BERKSHIRE HATHAWAY INC      Buy   

                     Security   Shares   Price        Total  
0         Xerox Holdings Corp   650698  $16.60  $10,801,587  
1         Xerox Holdings Corp   456651  $16.27   $7,429,712  
2         Xerox Holdings Corp   706086  $16.07  $11,346,802  
3  Camping World Holdings Inc   500000  $40.18  $20,091,850  
4        BANK OF AMERICA CORP  3754220  $24.97  $93,729,734  


Get Activity data

This returns the top 10 stocks that the super investors have bought or sold. THis does not specify buying from selling.

In [53]:
def basicActivity(count):          
    df = pd.read_html('https://www.dataroma.com/m/allact.php?typ=a')
    df[0].columns = ['Portfolio Manager - Firm', 'Period', '1','2', '3', '4','5', '6', '7','8', '9', '10']
    return Counter(list(df[0].iloc[:,2:].stack())).most_common(count)

In [54]:
ba = basicActivity(10)

In [55]:
print(ba)

[('FB', 13), ('GOOGL', 12), ('BRK.B', 10), ('GOOG', 10), ('AMZN', 7), ('V', 6), ('MSFT', 6), ('JPM', 6), ('JNJ', 5), ('UNH', 5)]
